In [ ]:
# Analyse des données de saturation

In [13]:
os.getcwd()[:-34]

'C:\\Users\\phili\\github\\'

In [14]:
import os
import sys
new_path = os.getcwd()[:-34] + 'qualicharge-rtet'
sys.path.append(new_path)

In [15]:
import json, copy
from shapely import LineString, Point
import numpy as np
import geopandas as gpd
import pandas as pd
import geo_nx as gnx 
import networkx as nx 

from geo_nx import geom_to_crs, cast_id
from qualicharge_rtet import proximite, insertion_projection, association_stations
from qualicharge_rtet import creation_pandas_stations, Afir, export_stations_parcs
from qualicharge_rtet import get_rtet_attr_station, get_parc_id_station, propagation_attributs_core

In [66]:
GEOM = "geometry"
NODE_ID = "node_id"
NATURE = "nature"
WEIGHT = "weight"
CORE = "core"

data_quali = '../data/'
data_rtet = '../../rtet/'
origine = 'V1_0_0_'
stations_parcs_file = 'stations_parcs.geojson'
pdc_activite_file = ['pdc_en_activite_paca_avril_prod.json', 'pdc_en_activite_paca_avril_staging.json']
sessions_file = ['session_bouches_du_rhone_25_avril_staging.json', 'session_bouches_du_rhone_25_avril_prod.json']
status_file = ['status_bouches_du_rhone_25_avril_AM_staging.json', 'status_bouches_du_rhone_25_avril_AM_prod.json',
               'status_bouches_du_rhone_25_avril_PM18_staging.json', 'status_bouches_du_rhone_25_avril_PM24_staging.json',
               'status_bouches_du_rhone_25_avril_PM24_prod.json',]
refnat = {'tiles': 'cartodbpositron', 'location': [46.3, 2.3], 'zoom_start': 7}

In [67]:
stations_parcs = gpd.read_file(data_rtet + origine + stations_parcs_file)
#stations_parc

In [69]:
liste_status = [pd.read_json(data_quali + file, orient='records') for file in status_file]
status = pd.concat(liste_status , ignore_index=True)
status['horodatage'] = pd.to_datetime(status['horodatage'], format='ISO8601').astype('datetime64[s, UTC+02:00]')
#status

In [70]:
liste_sessions = [pd.read_json(data_quali + file, orient='records') for file in sessions_file]
sessions = pd.concat(liste_sessions , ignore_index=True)
sessions['start'] = pd.to_datetime(sessions['start'], format='ISO8601').astype('datetime64[s, UTC+02:00]')
sessions['end'] = pd.to_datetime(sessions['end'], format='ISO8601').astype('datetime64[s, UTC+02:00]')
#sessions

In [71]:
liste_pdc = [pd.read_json(data_quali + file, orient='records') for file in pdc_activite_file]
pdc_en_activite = pd.concat(liste_pdc , ignore_index=True)
#pdc_en_activite

In [98]:
# date_range
per = pd.Series(range(20)) / 2
periode = pd.DataFrame( {'periode': per})

test = pd.DataFrame( {'date': [1, 1.2, 3, 3.5, 5, 6.1, 12],
                      'status':['a', 'b', 'c', 'c', 'b', 'a', 'c'],
                      'pdc': ['p1', 'p2', 'p2', 'p1', 'p2', 'p1', 'p2']})
init = pd.DataFrame( {'date': [-1] * 2, 'status': ['d'] * 2, 'pdc': ['p1', 'p2']}) 

test_ord = pd.concat([test, init]).sort_values(by=['pdc', 'date'])
test_ord['datef'] = list(test_ord['date'])[1:len(test_ord)] + [10000]
test_ord['pdcf'] = list(test_ord['pdc'])[1:len(test_ord)] + ['aucun']
test_ord

,date,status,pdc,datef,pdcf
0,-1.0,d,p1,1.0,p1
0,1.0,a,p1,3.5,p1
3,3.5,c,p1,6.1,p1
5,6.1,a,p1,-1.0,p2
1,-1.0,d,p2,1.2,p2
1,1.2,b,p2,3.0,p2
2,3.0,c,p2,5.0,p2
4,5.0,b,p2,12.0,p2
6,12.0,c,p2,10000.0,aucun


In [99]:
test_merged = pd.merge(test_ord, periode, how='cross')
test_merged[((test_merged['pdc'].eq(test_merged['pdcf'])) &
             (test_merged['periode'] >= test_merged['date']) &
             (test_merged['periode'] < test_merged['datef'])) |
            (~(test_merged['pdc'].eq(test_merged['pdcf'])) &
             (test_merged['periode'] >= test_merged['date']))][['periode', 'status', 'pdc']]

,periode,status,pdc
0,0.0,d,p1
1,0.5,d,p1
22,1.0,a,p1
23,1.5,a,p1
24,2.0,a,p1
25,2.5,a,p1
26,3.0,a,p1
47,3.5,c,p1
48,4.0,c,p1
49,4.5,c,p1
